# Tesis

Este es el cuaderno de Jupyter para la implementación de la tesis. <br >
Este código utiliza el trabajo realizado por Diego Gomez (Poner aquí link al artículo de el, o el github o lo que sea) y lo aplica al problema de la regulación del nivel de glucosa en la sangre en pacientes Diabéticos Tipo 1. <br >
Se utiliza la librería de "simglucose" que permite simular la dinámica de la glucosa, de acuerdo con el simulador aprobado por la FDA de UVa/Padova (Versión 2008). <br >
Link de la librería de simglucose: https://github.com/jxx123/simglucose <br >
Link del GitHub de Diego: https://github.com/tarod13 <br >

# Tareas
1. Entender bien la librería de simglucose: rewards, acciones, estados, todo eso. (En progreso: ~5%) 
2. Hacer lo del mapeo de estados a acciones: s->a (Sin comenzar: 0%)
3. Hacer lo del mapeo de estados a Habilidades: s->A (Sin comenzar: 0%)
4. Implementar lo de Diego para encontrar los conceptos: s->S (Sin comenzar: 0%)
5. Analizar todo

In [1]:
import numpy as np
import simglucose